# Animación realizada con *PyGame*

In [8]:
import numpy as np
import pygame
import sys
import os

Se crea una función que recibe el *path* del archivo .txt generado en ```ammonia_test.ipynb``` y se utiliza ```pygame``` para mostrar la evolución del tanque a lo largo del tiempo.

In [ ]:
def animate_tank(path):

    # --------- Paso 1: Leer y procesar los datos del archivo -----------------
    # print(path.split('_'))
    mass_rate = path.split('_')[5]

    with open(path, 'r') as file:
        data = file.readlines()

    data = data[1:]
    cleaned_data = []
    for element in data:
        cleaned_line = element.strip().split(',')
        cleaned_data.append(cleaned_line)

    time_list = np.array([float(i) for i in cleaned_data[0]])
    V_L_list = np.array([float(i) for i in cleaned_data[6]])
    Q_tot_list = np.array([float(i) for i in cleaned_data[13]])
    d_i = float(cleaned_data[14][0])
    V_tank = float(cleaned_data[16][0])


    tank_area = np.pi * (d_i ** 2) / 4
    max_height = V_tank / tank_area  # m
    liquid_heights = [V / tank_area for V in V_L_list]  # m

    # --------- Paso 2: Configurar Pygame --------------------------------------
    
    pygame.init()

    # Ventana
    WIDTH, HEIGHT = 1200, 800
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("Animación de nivel de líquido")

    # Colores
    GRAY = (180, 180, 180)
    WHITE = (255, 255, 255)
    BLUE = (173, 216, 230)
    PINK = (255, 220, 220)
    BLACK = (0, 0, 0)
    ORANGE = (255, 165, 0)

    # Create a Font object
    font = pygame.font.SysFont('Arial', 16) # Using a system font, size 16

    # Geometría del tanque
    x_tank, y_tank, w_tank, h_tank = 425, 200, (240 + 120), (240 + 126)

    # Datos de simulación (ejemplo)
    time = time_list                 # 10 s simulados
    height_liquid = liquid_heights  # ejemplo: sube asintóticamente

    # Configurar reloj para controlar velocidad
    clock = pygame.time.Clock()
    fps = 240

    # Escalar alturas a píxeles dentro del tanque
    liquid_min = y_tank + h_tank - 20   # fondo
    liquid_max = y_tank + 80            # parte superior del líquido

    def get_liquid_y(h_fraction):
        # Convierte la fracción (0 a 1) en coordenada vertical
        return liquid_min - h_fraction * (liquid_min - liquid_max)

    running = True
    frame_count = 0
    # Create a directory to store frames if it doesn't exist
    if not os.path.exists("frames"):
        os.makedirs("frames")

    # Loop principal de animación
    for i in range(len(time_list)):
        screen.fill(WHITE)

        liquid_fraction = (V_L_list[i] / V_tank) * 100
        vapor_fraction = 100 - liquid_fraction

        # Render the text
        text_surface = font.render(f'{V_tank} m^3 Tank filling at {time_list[i]/3600:.2f} hr', True, BLACK) # Text, antialias, color
        text_surface2 = font.render(rf'Flow rate: m_L = {mass_rate} kg/s', True, BLACK) # Text, antialias, color

        # Textos para indicar el volumen de líquido y vapor
        text_surface3 = font.render(rf'Lf = {liquid_fraction:.2f}%', True, BLACK) # Text, antialias, color
        text_surface4 = font.render(rf'Vf = {vapor_fraction:.2f}%', True, BLACK) # Text, antialias, color

        # Get the rectangle of the text surface for positioning
        text_rect = text_surface.get_rect()
        text_rect.center = (10 + WIDTH // 2, 20) # Center the text

        text_rect2 = text_surface2.get_rect()
        text_rect2.center = (10 + WIDTH // 2, 40) # Center the text
        
        screen.blit(text_surface2, text_rect2) # Blit the text to the screen
        screen.blit(text_surface, text_rect) # Blit the text to the screen

        # --- Tanque ---
        pygame.draw.rect(
            screen, GRAY, (x_tank, y_tank, w_tank, h_tank),
        )

        # Rectángulo interior del tanque
        pygame.draw.rect(
            screen, BLACK, (x_tank+20, y_tank+20, w_tank-40, h_tank-40))

    # Interior del tanque (zona donde realmente hay líquido)
        inner_x = x_tank + 20
        inner_y = y_tank + 20
        inner_w = w_tank - 40
        inner_h = h_tank - 40

        # Calcular altura del líquido (lineal, proporcional)
        h_frac = V_L_list[i] / V_tank   # 0–1
        liquid_height_px = h_frac * inner_h

        # Coordenada del nivel superior del líquido
        y_liquid = inner_y + (inner_h - liquid_height_px)

        # Dibujar líquido
        liquid_rect = pygame.Rect(inner_x, y_liquid, inner_w, liquid_height_px)
        pygame.draw.rect(screen, BLUE, liquid_rect)

        # Dibujar vapor
        vapor_rect = pygame.Rect(inner_x, inner_y, inner_w, (y_liquid - inner_y))
        pygame.draw.rect(screen, PINK, vapor_rect)

        # --- Línea de separación --- 
        pygame.draw.line(screen, BLACK, (x_tank+20, y_liquid), (x_tank+w_tank-20, y_liquid), 2)

        # Gráfico de los porcentajes de líquido y vapor
        text_rect3 = text_surface3.get_rect()
        text_rect3.center = (10 + WIDTH // 2, 475) # Center the text

        text_rect4 = text_surface4.get_rect()
        text_rect4.center = (10 + WIDTH // 2, 250) # Center the text

        screen.blit(text_surface4, text_rect4) # Blit the text to the screen
        screen.blit(text_surface3, text_rect3) # Blit the text to the screen
        
        # Graficar una flecha proporcional al calor total Q_tot_list (apuntando hacia el tanque)

        Q_tot = Q_tot_list[i]  # W
        arrow_length = min(max(Q_tot / 1000, 20), 200)  # Largo entre 20 y 200 px

        # Flecha horizontal apuntando a la izquierda
        arrow_start = (800, 500)                 # Punto derecho
        arrow_end = (800 - arrow_length, 500)    # Punto izquierdo

        # Línea principal
        pygame.draw.line(screen, ORANGE, arrow_start, arrow_end, 5)

        # Punta de la flecha (triángulo)
        pygame.draw.polygon(screen, ORANGE, [
            (arrow_end[0] + 10, arrow_end[1] - 10),   # arriba
            (arrow_end[0] + 10, arrow_end[1] + 10),   # abajo
            (arrow_end[0] - 10, arrow_end[1])         # punta izquierda
        ])

        # Texto al lado de la flecha
        heat_text = font.render(r'Q_tot = {:.1f} kW'.format(Q_tot / 1e3), True, BLACK)
        heat_text_rect = heat_text.get_rect()
        heat_text_rect.center = (arrow_start[0] - arrow_length / 2, arrow_start[1] - 30)
        screen.blit(heat_text, heat_text_rect)


        # Actualizar pantalla
        pygame.display.flip()
        clock.tick(fps)

        # Permitir salir del programa
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

    pygame.quit()


In [ ]:
path_10_kg_positive = 'data/ammonia_tank_full_data_mL_10.0_kg_per_s.txt'
animate_tank(path_10_kg_positive)

KeyboardInterrupt: 

: 

Nota: Se guardó como *.gif* una animación realizada para un tanque de $6.75\;[L]$ para un flujo másico de $\dot{m}_{L} = 25\cdot 10^{-6}\;[\frac{kg}{s}]$, el cual se encuentra en el archivo ```animated_pygame.gif```